# How to Train YOLOv9 on a Custom Dataset
---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/WongKinYiu/yolov9)
[![arXiv](https://img.shields.io/badge/arXiv-2402.13616-b31b1b.svg)](https://arxiv.org/pdf/2402.13616.pdf)

## Before you start &rarr; Update

Let's update our Github repo to have our latest changes for the training.

In [ ]:
import os

HOME = os.path.abspath(os.path.join(os.getcwd(), '../..'))
YOLO = os.path.join(HOME, 'yolov9')
print(HOME)
print(YOLO)

!cd {YOLO}
!git reset --hard
!git pull
!cd {HOME}

If the update was successfull and the notebook is reloaded with the new changes we install the needed packages for our container. After this it is recommended to also restart the kernel. After this final step you can directly start from the next heading!

In [ ]:
pip install seaborn thop

After this process it is recommended to reload the kernel. \
Meaning after the **RELOAD** you can continue below this cell!

## Start after successfull updates and package installation

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that.

In [ ]:
!nvidia-smi

Mon May 27 16:11:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   35C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` and `YOLO` constant.

In [ ]:
import os, shutil
from IPython.display import Image


# This needs to be adapted depending on how what dir is mounted in the container
HOME = os.path.abspath(os.path.join(os.getcwd(), '../..'))
YOLO = os.path.join(HOME, 'yolov9')
print(HOME)
print(YOLO)

Set environment variables for PyTorch: `LOCAL_RANK` and `RANK` have to be set to `='-1'` in order to not trigger distributed GPU training.

In [ ]:
os. environ['LOCAL_RANK'] = '-1'
os.environ['RANK'] = '-1'

print(f"LOCAL_RANK = {os.environ.get('LOCAL_RANK')}")
print(f"RANK = {os.environ.get('RANK')}")

Number the trainign runs consecutively

In [ ]:
if os.path.exists(f"{YOLO}/runs"):
    shutil.rmtree(os.path.join(YOLO, 'runs'))

Lauf_num = 'Lauf_'

Check if weights and data is downloaded

In [ ]:
!ls -la {HOME}/weights
!ls -la {HOME}/loco

## Train custom model on LOCO dataset

**Note:** `train.py` is only for gelan models `train_dual.py` is only for yolov9 models

In [ ]:
!python3 {YOLO}/train_dual.py --batch 8 --epochs 50 --device 0 --min-items 0 --img 640 \
--name Lauf_ \
--data {YOLO}/loco.yaml \
--weights {HOME}/weights/yolov9-e.pt \
--cfg {YOLO}/models/detect/yolov9_custom.yaml \
--hyp hyp.scratch-high.yaml

## Detection with self-trained detection model

**Note:** There is no test set. Image `509189,8734.jpg` is just one example from the training dataset. It includes a lot of pallets and a pallet truck which should be detected.

In [ ]:
!python3 {YOLO}/detect_dual.py --weights {YOLO}/runs/train/Lauf_/weights/best.pt --conf 0.1 --source {HOME}/loco/images/val/509189,8734.jpg --device 0

### Display predicted images

In [ ]:
Image(filename=f"{HOME}/loco/images/val/509189,8734.jpg", width=640)    # or maybe width=640???

In [ ]:
Image(filename=f"{YOLO}/runs/detect/Lauf_/509189,8734.jpg", width=640)    # or maybe width=640???

## Examine Training Results

**NOTE:** By default, the results of each subsequent training sessions are saved in `{HOME}/yolov9/runs/train/`, in directories named `exp`, `exp2`, `exp3`, ... You can override this behavior by using the `--name` parameter.

For this Notebook I used `'Lauf_'` as naming, which is also already accounted for in the follwoing code.

In [ ]:
!ls {HOME}/yolov9/runs/train/Lauf_/

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/Lauf_/results.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/Lauf_/confusion_matrix.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/Lauf_/val_batch0_pred.jpg", width=1000)

## Validate Custom Model

**NOTE**: Look into this! Why do you need validation?

In [ ]:
%cd {HOME}/yolov9

!python val_dual.py \
--img 640 --batch 8 --conf 0.001 --iou 0.7 --device 0 \
--data {YOLO}/loco.yaml \
--weights {YOLO}/runs/train/Lauf_/weights/best.pt

## BONUS: Deploy YOLOv9 Model with Inference

**NOTE:** To deploy the model and display inference results, we will need two additional packages - [`inference`](https://pypi.org/project/inference) and [`supervision`](https://pypi.org/project/supervision). Let's install and import them!

**REMARK** Maybe check the original Colab ;)